Bharath Gunasekaran

CMPE 258 

Assignment 5 Part D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split 

plt.style.use('seaborn-whitegrid')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/CMPE-258/HW5/new-york-city-taxi-fare-prediction/train.csv', nrows=1000000)


In [ ]:
df = df[df.fare_amount >0]
df = df.dropna()

In [ ]:
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,999923.000000,999923.000000,999923.000000,999923.000000,999923.000000,999923.000000
mean,11.348945,-72.526826,39.929108,-72.528356,39.920225,1.684955
std,9.821201,12.057522,7.626026,11.323010,8.200913,1.323921
min,0.010000,-3377.680935,-3116.285383,-3383.296608,-3114.338567,0.000000
25%,6.000000,-73.992060,40.734965,-73.991385,40.734047,1.000000
50%,8.500000,-73.981792,40.752695,-73.980135,40.753166,1.000000
75%,12.500000,-73.967095,40.767154,-73.963654,40.768129,2.000000
max,500.000000,2522.271325,2621.628430,45.581619,1651.553433,208.000000


# Feature Engineering

In [ ]:
def calculate_distance(features):
    features['distance'] = ((features['pickup_latitude'] - features['dropoff_latitude'])**2 +  (features['pickup_longitude'] - features['dropoff_longitude'])**2)**0.5
    return features

df = calculate_distance(df)

In [ ]:
def parse_date(features):
  features['date'] = pd.to_datetime(features['pickup_datetime'].str[:19])
  features['hour'] = features['date'].dt.hour
  features['month'] = features['date'].dt.month
  features['day'] = features['date'].dt.weekday
  return features.drop(columns=['date','pickup_datetime'])

df = parse_date(df)

In [ ]:
df= df[df.passenger_count <= 7]

In [ ]:
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,hour,month,day
count,999922.000000,999922.000000,999922.000000,999922.000000,999922.000000,999922.000000,999922.000000,999922.000000,999922.000000,999922.000000
mean,11.348954,-72.526899,39.929147,-72.528428,39.920265,1.684748,0.231035,13.509534,6.268018,3.039860
std,9.821203,12.057310,7.625926,11.322784,8.200820,1.307746,11.282671,6.513776,3.436241,1.949961
min,0.010000,-3377.680935,-3116.285383,-3383.296608,-3114.338567,0.000000,0.000000,0.000000,1.000000,0.000000
25%,6.000000,-73.992060,40.734965,-73.991385,40.734047,1.000000,0.012401,9.000000,3.000000,1.000000
50%,8.500000,-73.981792,40.752696,-73.980135,40.753166,1.000000,0.021444,14.000000,6.000000,3.000000
75%,12.500000,-73.967095,40.767154,-73.963654,40.768129,2.000000,0.038403,19.000000,9.000000,5.000000
max,500.000000,2522.271325,2621.628430,45.581619,1651.553433,6.000000,6933.843499,23.000000,12.000000,6.000000


# Spliting data into feature and target

In [ ]:
X = df.drop(columns=['key','fare_amount'])
Y = df['fare_amount']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state = 42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(799937, 9)
(199985, 9)
(799937,)
(199985,)


# Running XGBoost Model

In [ ]:
import xgboost as xgb


In [ ]:
params = {
    'max_depth': 7,
    'gamma' :0,
    'eta':.03, 
    'subsample': 1,
    'colsample_bytree': 0.9, 
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'silent': 0
}

In [ ]:
def XGBmodel(X_train,X_test,y_train,y_test,params):
    matrix_train = xgb.DMatrix(X_train,label=y_train)
    matrix_test = xgb.DMatrix(X_test,label=y_test)
    model=xgb.train(params=params,
                    dtrain=matrix_train,num_boost_round=5000, 
                    early_stopping_rounds=10,evals=[(matrix_test,'test')])
    return model

model = XGBmodel(X_train,X_test,y_train,y_test,params)

[02:57:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	test-rmse:14.2768
Will train until test-rmse hasn't improved in 10 rounds.
[1]	test-rmse:13.921
[2]	test-rmse:13.5776
[3]	test-rmse:13.2247
[4]	test-rmse:12.9048
[5]	test-rmse:12.5751
[6]	test-rmse:12.2562
[7]	test-rmse:11.9482
[8]	test-rmse:11.651
[9]	test-rmse:11.3637
[10]	test-rmse:11.0872
[11]	test-rmse:10.82
[12]	test-rmse:10.5618
[13]	test-rmse:10.3133
[14]	test-rmse:10.074
[15]	test-rmse:9.8433
[16]	test-rmse:9.62114
[17]	test-rmse:9.42024
[18]	test-rmse:9.21387
[19]	test-rmse:9.01483
[20]	test-rmse:8.82355
[21]	test-rmse:8.64038
[22]	test-rmse:8.4747
[23]	test-rmse:8.31418
[24]	test-rmse:8.14912
[25]	test-rmse:7.9906
[26]	test-rmse:7.83875
[27]	test-rmse:7.70361
[28]	test-rmse:7.56254
[29]	test-rmse:7.42832
[30]	test-rmse:7.30006
[31]	test-rmse:7.17678
[32]	test-rmse:7.05821
[33]	test-rmse:6.94445
[34]	test-rmse:6.83572
[35]	test-rmse:6.73195
[36]	t